# Valorant Champions 2025: Stats Scraper

## Objective

The goal of this notebook is to programmatically extract player statistics from the VALORANT Champions 2025 event page on VLR.gg. The script automates the process of data collection and structures it into a clean format for further analysis.


## Tools & Libraries

This project leverages a standard stack for web scraping and data manipulation in Python:

* **Requests:** To perform HTTP requests and fetch the HTML content of the web page.
* **BeautifulSoup:** For parsing the HTML and navigating the document's structure to locate the data table.
* **Pandas:** To organize the extracted data into a structured DataFrame, which is ideal for analysis and exporting.


## The Process: Step-by-Step

The notebook follows a clear, four-step process to extract the data.

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd


### 1. Fetch Web Page Content
First, we define the target URL and use the `requests` library to send a GET request. A `User-Agent` header is included to simulate a browser, ensuring a successful response from the server.

In [ ]:
url = "https://www.vlr.gg/event/stats/2283/valorant-champions-2025"
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36'
}

In [ ]:
try:
    response = requests.get(url, headers=headers)
    response.raise_for_status()
except requests.exceptions.RequestException as e:
    print(f"Erro ao acessar a URL: {e}")
    exit()


### 2. Parse HTML & Locate Table
Once we have the HTML, `BeautifulSoup` is used to parse it. We then specifically search for the `<table>` element that contains the statistics, identifying it by its unique CSS classes (`'wf-table mod-stats mod-scroll'`).

In [ ]:
soup = BeautifulSoup(response.content, 'html.parser')
stats_table = soup.find('table', class_='wf-table mod-stats mod-scroll')

### 3. Iterate and Extract Data
With the table located, the script iterates through each row (`<tr>`) of the table body:
* It finds the player's name and team within the first cell.
* It loops through the remaining cells (`<td>`) to capture each statistic.
* It handles special cases where data is nested within `<span>` tags.
* All extracted data for each player is stored as a list.

In [ ]:
if stats_table:
    table_headers = [header.get_text(strip=True) for header in stats_table.find_all('th')]

    rows = []
    for row in stats_table.find('tbody').find_all('tr'):
        player_cell = row.find('td', class_='mod-player')
        player_name_div = player_cell.find('div', class_='text-of')
        player_team_div = player_cell.find('div', class_='stats-player-country')

        player = player_name_div.get_text(strip=True) if player_name_div else ''
        team = player_team_div.get_text(strip=True) if player_team_div else ''

        row_data = [(player + ' ' + team).strip()]

        other_cells = row.find_all('td')[1:]
        for cell in other_cells:
            span_val = cell.find('span', class_='stats-sq')
            if span_val:
                row_data.append(span_val.get_text(strip=True))
            else:
                row_data.append(cell.get_text(strip=True))

        rows.append(row_data)

### 4. Create a Structured DataFrame
Finally, the list of rows is passed to the `pandas` library, along with the extracted table headers, to create a clean and organized DataFrame.

In [ ]:
df = pd.DataFrame(rows, columns=table_headers)

In [ ]:
df.head(10)

,Player,Agents,Rnd,R2.0,ACS,K:D,KAST,ADR,KPR,APR,...,FDPR,HS%,CL%,CL,KMax,K,D,A,FK,FD
0,Chronicle FNC,,43,1.65,269.5,2.10,84%,185.9,1.02,0.33,...,0.05,34%,,0/4,25,44,21,14,6,2
1,something PRX,,40,1.60,283.0,1.74,88%,187.5,1.00,0.53,...,0.03,20%,40%,2/5,21,40,23,21,6,1
2,aspas MIBR,,37,1.54,270.5,2.06,81%,181.9,0.95,0.27,...,0.03,30%,,0/3,22,35,17,10,6,1
3,Verno MIBR,,37,1.44,244.5,2.19,86%,155.7,0.95,0.27,...,0.05,23%,50%,1/2,18,35,16,10,5,2
4,Boaster FNC,,43,1.44,221.0,1.64,77%,134.9,0.84,0.30,...,0.02,52%,50%,2/4,24,36,22,13,1,1
5,f0rsakeN PRX,,40,1.36,268.0,1.64,78%,164.8,1.03,0.43,...,0.18,37%,25%,1/4,23,41,25,17,6,7
6,Cloud GX,,58,1.34,253.7,1.42,71%,158.2,0.88,0.26,...,0.09,26%,,0/9,23,51,36,15,6,5
7,RieNs TH,,46,1.32,241.5,1.41,85%,163.7,0.89,0.41,...,0.04,34%,13%,1/8,21,41,29,19,4,2
8,Jinggg PRX,,40,1.32,286.0,1.44,78%,182.7,0.98,0.30,...,0.13,16%,,0/4,24,39,27,12,6,5
9,MaKo DRX,,44,1.31,207.5,1.70,86%,133.8,0.77,0.43,...,0.07,41%,33%,2/6,17,34,20,19,3,3


## Final Outcome & Next Steps

The script successfully produces a **pandas DataFrame** containing all player statistics from the event.

This structured data is now ready for the next phase of a data project:
* **Data Cleaning:** Converting columns to appropriate numeric types.
* **Exporting:** Saving the DataFrame to a `.csv` file.
* **Analysis & Visualization:** Using the data to build dashboards (in tools like Power BI) and uncover insights into player performance.